In [1]:
from linkedin_api import Linkedin

In [2]:
api = Linkedin('stand-with-palestine@outlook.com', 'Palestine1948')

In [74]:
post_url="https://www.linkedin.com/posts/stand-with-palestine_standwithpalestine-stopgenocide-stopisrael-activity-7240320989366992898-rOYo/?utm_source=share&utm_medium=member_desktop"

# https://www.linkedin.com/posts/stand-with-palestine_standwithpalestine-stopgenocide-stopisrael-activity-7240079286014001153-YmaF/?utm_source=share&utm_medium=member_desktop
post_urn="7240079286014001153"
post_urn="7240056637007282177"
comments=api.get_post_comments(post_urn=post_urn)

In [75]:
comments[0].keys()

dict_keys(['commenterProfileId', 'dashEntityUrn', 'pinned', 'edited', 'index', 'commenterForDashConversion', 'timeOffset', 'originalLanguage', 'rootSocialPermissions', 'commenter', 'socialDetail', 'urn', 'threadId', 'entityUrn', 'commentV2', 'createdTime', 'canDelete', 'comment', 'permalink', 'actions', 'contributed', 'trackingId'])

In [84]:
comments[0]

{'commenterProfileId': 'ACoAAC7FrgYB0IDpQv4t94kmRRRHcJTADomKjSk',
 'dashEntityUrn': 'urn:li:fsd_comment:(7240089505125343232,urn:li:ugcPost:7240056636231294977)',
 'pinned': False,
 'edited': True,
 'index': -1,
 'commenterForDashConversion': {'urn': 'urn:li:member:784707078',
  'image': {'attributes': [{'sourceType': 'PROFILE_PICTURE',
     'miniProfile': {'firstName': 'Anne',
      'lastName': 'Reeve',
      'dashEntityUrn': 'urn:li:fsd_profile:ACoAAC7FrgYB0IDpQv4t94kmRRRHcJTADomKjSk',
      'occupation': 'Anne Reeve Author, Chairperson Writers ‘ Group ,Animal Advocate, Poet,',
      'objectUrn': 'urn:li:member:784707078',
      'entityUrn': 'urn:li:fs_miniProfile:ACoAAC7FrgYB0IDpQv4t94kmRRRHcJTADomKjSk',
      '$anti_abuse_metadata': {'$anti_abuse_uuid': '8bf6086f-5bc8-4ac5-a0df-83133191cfb2'},
      'backgroundImage': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 800,
          'fileIdentifyingUrlPathSegment': '200_800/profile-displaybackgroundimage-shrink_200_800/0/

In [93]:
c=0 # comment number
comments[c]['comment']['values'][0]['value']

'And still they use mass destruction of babies \n\nWhy is there no CEASEFIRE .? Children are \n\nContinuing to be immunised for Polio 👐🏾🤲🏼\n\nPalestine 🇵🇸🩸is bleeding 🩸 stop bombs 💣 \n\nI stand with Palestine 🇵🇸 freedom peace ☮️\n\nWake up world 🌎 ASAP NOW SOS 🆘'

In [90]:
i=0 # reply number
comments[0]['socialDetail']['comments']['elements'][i]["commentV2"]['text']

'Anne Reeve We were never able to understand this. Thank you so much for your voice.\n'

In [89]:
i=1
comments[0]['socialDetail']['comments']['elements'][i]['comment']['values'][1]['value']

' Thank you, SWP.'

In [ ]:
"?commentUrn=urn%3Ali%3Acomment%3A%28ugcPost%3A7240079285212827648%2C7240331666982866944%29& 


dashCommentUrn=urn%3Ali%3Acomment%3A%28ugcPost%3A7240079285212827648%2C7240331666982866944%29&


actorCompanyId=99380150"

In [20]:
print(comments[0])

{'commenterProfileId': 'ACoAAAgEc6ABRkky1hofUfSkj2OJATmTBUrlwdo', 'dashEntityUrn': 'urn:li:fsd_comment:(7240329572053876736,urn:li:ugcPost:7240079285212827648)', 'pinned': False, 'edited': False, 'index': -1, 'commenterForDashConversion': {'urn': 'urn:li:member:134509472', 'image': {'attributes': [{'sourceType': 'PROFILE_PICTURE', 'miniProfile': {'firstName': 'Or', 'lastName': 'Lapid', 'dashEntityUrn': 'urn:li:fsd_profile:ACoAAAgEc6ABRkky1hofUfSkj2OJATmTBUrlwdo', 'occupation': 'Snr. Field Apps Engineer at Micron Technology', 'objectUrn': 'urn:li:member:134509472', 'entityUrn': 'urn:li:fs_miniProfile:ACoAAAgEc6ABRkky1hofUfSkj2OJATmTBUrlwdo', '$anti_abuse_metadata': {'$anti_abuse_uuid': '4aca8edc-eba3-446d-aea8-02d33acf71d3'}, 'backgroundImage': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 800, 'fileIdentifyingUrlPathSegment': '200_800/profile-displaybackgroundimage-shrink_200_800/0/1698991294976?e=1731542400&v=beta&t=Q0EljcZZvFBK7J86IQ1hdLqP5ArtsfI6xcXsVcmtS3E', 'expires